# Insurance Premium Modelling

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 

In [2]:
import arff
data_freq = arff.load('freMTPL2freq.arff') 
CONTRACTS = pd.DataFrame(data_freq, columns=["IDpol", "ClaimNb", "EXPOSURE", "Area", "POWER", "AGECAR","AGEDRIVER", "BonusMalus", "BRAND", "GAZ", "DENSITY", "REGION"])
CONTRACTS.head()
CONTRACTS.info()
CONTRACTS.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDpol       678013 non-null  float64
 1   ClaimNb     678013 non-null  float64
 2   EXPOSURE    678013 non-null  float64
 3   Area        678013 non-null  object 
 4   POWER       678013 non-null  float64
 5   AGECAR      678013 non-null  float64
 6   AGEDRIVER   678013 non-null  float64
 7   BonusMalus  678013 non-null  float64
 8   BRAND       678013 non-null  object 
 9   GAZ         678013 non-null  object 
 10  DENSITY     678013 non-null  float64
 11  REGION      678013 non-null  object 
dtypes: float64(8), object(4)
memory usage: 62.1+ MB


,IDpol,ClaimNb,EXPOSURE,POWER,AGECAR,AGEDRIVER,BonusMalus,DENSITY
count,6.780130e+05,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000
mean,2.621857e+06,0.053247,0.528750,6.454631,7.044265,45.499122,59.761502,1792.422405
std,1.641783e+06,0.240117,0.364442,2.050906,5.666232,14.137444,15.636658,3958.646564
min,1.000000e+00,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,1.000000
25%,1.157951e+06,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,92.000000
50%,2.272152e+06,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,393.000000
75%,4.046274e+06,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,1658.000000
max,6.114330e+06,16.000000,2.010000,15.000000,100.000000,100.000000,230.000000,27000.000000


# Calculating the average annualized frequency and its empirical variance.
#### Actual number of claims in the database Yi
#### Ei is the exposure
#### Average annualized frequency m_N
#### empirical variance S^2_N

![Steps average annualized frequency and its empirical variance](1.png)

In [3]:
import numpy as np

vY = CONTRACTS['ClaimNb']
vE = CONTRACTS['EXPOSURE']

m = np.sum(vY) / np.sum(vE)
v = np.sum((vY - m * vE) ** 2) / np.sum(vE)

average = m
variance = v
phi = v / m

print("average =", average, " variance =", variance, " phi =", phi)

average = 0.10070308464041304  variance = 0.10929293731914318  phi = 1.0852988039979359


### For a Poisson distribution, phi should be equal to 1.

# Considering the case where X is the region of the driver ( Categorical Variable)

![Steps average annualized frequency and its empirical variance](2.png)

In [4]:
vX = CONTRACTS['REGION']
unique_regions = vX.unique()

for region in unique_regions:
    vEi = CONTRACTS.loc[vX == region, 'EXPOSURE']
    vYi = CONTRACTS.loc[vX == region, 'ClaimNb']
    mi = sum(vYi) / sum(vEi)
    vi = sum((vYi - mi * vEi) ** 2) / sum(vEi)
    print("Region:", region)
    print("Average =", mi, " Variance =", vi, " Phi =", vi / mi)

Region: 'R82'
Average = 0.11096552573862725  Variance = 0.11711805706825394  Phi = 1.0554454303593228
Region: 'R22'
Average = 0.12281746438680714  Variance = 0.12700557992565698  Phi = 1.0341003257131216
Region: 'R72'
Average = 0.09411983738060346  Variance = 0.10241105240596311  Phi = 1.0880921095499931
Region: 'R31'
Average = 0.10227888604407133  Variance = 0.11352128736310269  Phi = 1.109919082558125
Region: 'R91'
Average = 0.10498025675054383  Variance = 0.15868028408893062  Phi = 1.511525014327121
Region: 'R52'
Average = 0.09163842181593423  Variance = 0.09651804401276  Phi = 1.0532486494216042
Region: 'R93'
Average = 0.10916359197663394  Variance = 0.12133488138842634  Phi = 1.1114958677284787
Region: 'R11'
Average = 0.13168417189998127  Variance = 0.1438617911228666  Phi = 1.092475952479199
Region: 'R24'
Average = 0.08960899455283504  Variance = 0.09340771459910474  Phi = 1.0423921735226023
Region: 'R94'
Average = 0.13986921610498568  Variance = 0.16709363111270994  Phi = 1.1946

### Assume that claims occurrence, for an insured, is driven by a homogeneous Poisson process, with intensity lamda


# Poisson Regression

![Steps average annualized frequency and its empirical variance](3.png)

In [5]:
#import numpy as np
import pandas as pd
from scipy.stats import poisson

Y = CONTRACTS['ClaimNb']
E = CONTRACTS['EXPOSURE']

# Calculate lambda (rate parameter of Poisson distribution)
lambda_val = np.sum(Y) / np.sum(E)
print("lambda:", lambda_val)

# Weighted mean calculation
weighted_mean = np.average(Y / E, weights=E)
print("Weighted Mean:", weighted_mean)

# Calculate probabilities for Poisson distribution for values 0 to 3
probabilities = poisson.pmf(np.arange(4), lambda_val) * 100
print("Probabilities (0 to 3):", probabilities)

# Here we find probabilities of 0 - 3 claims


lambda: 0.10070308464041304
Weighted Mean: 0.10070308464041304
Probabilities (0 to 3): [9.04201464e+01 9.10558766e+00 4.58480382e-01 1.53901296e-02]


In [6]:
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

# Select categorical columns to encode
categorical_cols = ['GAZ']
y = CONTRACTS['ClaimNb']

# Calculate offset (logarithm of EXPOSURE)
offset = np.log(CONTRACTS['EXPOSURE'])

# One-hot encode categorical columns
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(CONTRACTS[categorical_cols])

# Combine encoded categorical columns with numerical columns
X_numerical = CONTRACTS[['AGEDRIVER', 'DENSITY']]
X = np.concatenate((X_encoded, X_numerical), axis=1)

# Add constant term to the independent variables
X = sm.add_constant(X)

# Fit the GLM with Poisson family
reg = sm.GLM(y, X, family=sm.families.Poisson(), offset=offset).fit()

# Display summary
print(reg.summary())


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               678013
Model:                            GLM   Df Residuals:                   678009
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4665e+05
Date:                Mon, 22 Apr 2024   Deviance:                   2.2397e+05
Time:                        07:34:53   Pearson chi2:                 1.94e+06
No. Iterations:                     7   Pseudo R-squ. (CS):          0.0007521
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1415      0.018   -115.901      0.0

# A Poisson Regression to Model Yearly Claim Frequency

#### Categorising the age of the car in two classes: less than 15 years old, and more than 15 years old:

In [7]:
import pandas as pd

# Check the data type of the 'AGECAR' column
print(CONTRACTS['AGECAR'].dtype)

# If the data type is not numeric, convert it to numeric
CONTRACTS['AGECAR'] = pd.to_numeric(CONTRACTS['AGECAR'], errors='coerce')

# Check for any missing values after conversion
print(CONTRACTS['AGECAR'].isnull().sum())

# Now, apply the cut function
bins = [0, 15, float('inf')]  # Categories: 0-15, 16-inf
labels = ['0-15', '16-Inf']
CONTRACTS['AGECAR'] = pd.cut(CONTRACTS['AGECAR'], bins=bins, labels=labels, include_lowest=True)


float64
0


In [8]:
levels = pd.unique(CONTRACTS['AGECAR'])

print(levels)

['0-15', '16-Inf']
Categories (2, object): ['0-15' < '16-Inf']


# Considering only B12 brand of car

In [9]:
# Remove single quotes from the 'BRAND' column
CONTRACTS['BRAND'] = CONTRACTS['BRAND'].str.replace("'", "")

# Create a new column 'brandF' based on the condition 'BRAND == "B12"'
CONTRACTS['brandF'] = CONTRACTS['BRAND'].apply(lambda x: 'B12' if x == 'B12' else 'other')

# Convert the 'brandF' column to categorical type
CONTRACTS['brandF'] = pd.Categorical(CONTRACTS['brandF'], categories=['other', 'B12'])

# Display the first few rows to verify the changes
print(CONTRACTS.head())

   IDpol  ClaimNb  EXPOSURE Area  POWER AGECAR  AGEDRIVER  BonusMalus BRAND  \
0    1.0      1.0      0.10  'D'    5.0   0-15       55.0        50.0   B12   
1    3.0      1.0      0.77  'D'    5.0   0-15       55.0        50.0   B12   
2    5.0      1.0      0.75  'B'    6.0   0-15       52.0        50.0   B12   
3   10.0      1.0      0.09  'B'    7.0   0-15       46.0        50.0   B12   
4   11.0      1.0      0.84  'B'    7.0   0-15       46.0        50.0   B12   

       GAZ  DENSITY REGION brandF  
0  Regular   1217.0  'R82'    B12  
1  Regular   1217.0  'R82'    B12  
2   Diesel     54.0  'R22'    B12  
3   Diesel     76.0  'R72'    B12  
4   Diesel     76.0  'R72'    B12  


In [10]:
import statsmodels.api as sm
import pandas as pd

# Convert categorical variables into dummy variables
X = pd.get_dummies(CONTRACTS[['AGEDRIVER', 'AGECAR', 'DENSITY', 'brandF', 'POWER', 'GAZ']], drop_first=True)

# Add constant term to the independent variables
X = sm.add_constant(X)

# Calculate offset (logarithm of EXPOSURE)
offset = np.log(CONTRACTS['EXPOSURE'])

# Fit the GLM with Poisson family
regp = sm.GLM(CONTRACTS['ClaimNb'], X, family=sm.families.Poisson(), offset=offset).fit()

# Display summary
print(regp.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               678013
Model:                            GLM   Df Residuals:                   678006
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4606e+05
Date:                Mon, 22 Apr 2024   Deviance:                   2.2280e+05
Time:                        07:35:12   Pearson chi2:                 1.81e+06
No. Iterations:                     7   Pseudo R-squ. (CS):           0.002470
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.1607      0.025    -87.020

# Regression for Claims Data

In [11]:
data_sev = arff.load('freMTPL2sev.arff') 
CLAIMS = pd.DataFrame(data_sev, columns=["IDpol", "PurePremium"])
CLAIMS.head()
CLAIMS.info()
CLAIMS.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26639 entries, 0 to 26638
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IDpol        26639 non-null  float64
 1   PurePremium  26639 non-null  float64
dtypes: float64(2)
memory usage: 416.4 KB


,IDpol,PurePremium
count,2.663900e+04,2.663900e+04
mean,2.279864e+06,2.278536e+03
std,1.577202e+06,2.929748e+04
min,1.390000e+02,1.000000e+00
25%,1.087642e+06,6.868100e+02
50%,2.137413e+06,1.172000e+03
75%,3.180162e+06,1.228080e+03
max,6.113971e+06,4.075401e+06


In [12]:

CLAIMS = CLAIMS.groupby('IDpol')['PurePremium'].sum().reset_index()
# 24950 rows × 2 columns

In [13]:
# Perform an inner Join to merge the two dataframes
claims = pd.merge(CONTRACTS, CLAIMS, on='IDpol', how='inner')

In [14]:
claims.info

<bound method DataFrame.info of            IDpol  ClaimNb  EXPOSURE Area  POWER AGECAR  AGEDRIVER  BonusMalus  \
0          139.0      1.0      0.75  'F'    7.0   0-15       61.0        50.0   
1          190.0      1.0      0.14  'B'   12.0   0-15       50.0        60.0   
2          414.0      1.0      0.14  'E'    4.0   0-15       36.0        85.0   
3          424.0      2.0      0.62  'F'   10.0   0-15       51.0       100.0   
4          463.0      1.0      0.31  'A'    5.0   0-15       45.0        50.0   
...          ...      ...       ...  ...    ...    ...        ...         ...   
24939  6113521.0      1.0      0.18  'C'    4.0   0-15       26.0        60.0   
24940  6113793.0      1.0      0.14  'C'    7.0   0-15       51.0        50.0   
24941  6113817.0      1.0      0.17  'D'    4.0   0-15       35.0        51.0   
24942  6113834.0      2.0      0.17  'C'   15.0   0-15       36.0        50.0   
24943  6113971.0      1.0      0.13  'D'    7.0   0-15       39.0        50.0

# Log based Linear Regression with filtered Claim Amount/PurePremium

In [16]:
import statsmodels.api as sm
import pandas as pd

# Filter the data where PurePremium is less than 15000
claims_filtered = claims[claims['PurePremium'] < 15000]

# Linear regression model with log-transformed dependent variable
reg_logn = sm.OLS.from_formula('np.log(PurePremium) ~ AGECAR + GAZ', data=claims_filtered).fit()
print(reg_logn.summary())

                             OLS Regression Results                            
Dep. Variable:     np.log(PurePremium)   R-squared:                       0.000
Model:                             OLS   Adj. R-squared:                  0.000
Method:                  Least Squares   F-statistic:                     1.279
Date:                 Mon, 22 Apr 2024   Prob (F-statistic):              0.278
Time:                         07:35:50   Log-Likelihood:                -36466.
No. Observations:                24626   AIC:                         7.294e+04
Df Residuals:                    24623   BIC:                         7.296e+04
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            6.8590   

In [17]:
# gamma GLM model with log-transformed dependent variable

import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

# Filter the data where PurePremium is less than 15000
claims_filtered = claims[claims['PurePremium'] < 15000]

# Select predictor variables (X) and response variable (y)
X = claims_filtered[['AGECAR', 'GAZ']]
y = claims_filtered['PurePremium']

# One-hot encode categorical variables
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X)

# Convert encoded features to DataFrame
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(X.columns))

# Reset indices of X_encoded_df and y to ensure alignment
X_encoded_df.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

# Fit the gamma GLM model
reg_gamma = sm.GLM(y, sm.add_constant(X_encoded_df), family=sm.families.Gamma()).fit()

# Display summary for the gamma GLM model
print(reg_gamma.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:            PurePremium   No. Observations:                24626
Model:                            GLM   Df Residuals:                    24623
Model Family:                   Gamma   Df Model:                            2
Link Function:          inverse_power   Scale:                          1.2339
Method:                          IRLS   Log-Likelihood:            -2.0511e+05
Date:                Mon, 22 Apr 2024   Deviance:                       21757.
Time:                        07:35:55   Pearson chi2:                 3.04e+04
No. Iterations:                     8   Pseudo R-squ. (CS):          0.0003854
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0007   6.73e-06     99.753

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\ADMIN\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:298: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
